In [1]:
from pprint import pprint
from elasticsearch import Elasticsearch
import os
from dotenv import load_dotenv
import uuid
load_dotenv()

True

In [2]:

# pprint(os.getenv("ELASTIC_SEARCH_API_KEY"))

In [3]:
es = Elasticsearch(os.getenv("ELASTIC_SEARCH_URL"), api_key=os.getenv("ELASTIC_SEARCH_API_KEY"))

client_info = es.info()

pprint("Connected to elastic Search")
print(client_info)

'Connected to elastic Search'
{'name': 'serverless', 'cluster_name': 'fb96fd8be3e24d4c8a75e6aaf5802bf6', 'cluster_uuid': 'R11pXxymTzym4PV5vRNTYg', 'version': {'number': '8.11.0', 'build_flavor': 'serverless', 'build_type': 'docker', 'build_hash': '00000000', 'build_date': '2023-10-31', 'build_snapshot': False, 'lucene_version': '9.7.0', 'minimum_wire_compatibility_version': '8.11.0', 'minimum_index_compatibility_version': '8.11.0'}, 'tagline': 'You Know, for Search'}


In [4]:
es.indices.delete(index="myindex", ignore_unavailable=True)

# Not going to work on serverless
# es.indices.create(index="myindex", settings={
#     "index": {
#         "number_of_shards": 3,
#         "number_of_replica": 2
#     }
# })

index_name = "myindex"

es.indices.create(index=index_name)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'myindex'})

In [5]:
from datetime import datetime

pprint(datetime.now())

datetime.datetime(2025, 8, 31, 12, 49, 14, 701676)


In [6]:
# random_uuid = uuid.uuid4()
# # print(random_uuid)

first_document = {
    "document_id" : uuid.uuid4(),
    "title" : "Test Document 1",
    "author" : "Uday Yadav",
    "date" : datetime.now()
}

es.index(index=index_name, document=first_document)

ObjectApiResponse({'_index': 'myindex', '_id': '668sAJkB-JJed6hytyjz', '_version': 1, 'result': 'created', '_shards': {'total': 1, 'successful': 1, 'failed': 0}, '_seq_no': 0, '_primary_term': 1})

In [7]:
import json

dummy_data = json.load(open("./data/dummy_data.json"))
pprint(dummy_data)

[{'created_on': '2024-09-22',
  'text': 'This is the first sample document text.',
  'title': 'Sample Title 1'},
 {'created_on': '2024-09-24',
  'text': 'Here is another example of a document.',
  'title': 'Sample Title 2'},
 {'created_on': '2024-09-24',
  'text': 'The content of the third document goes here.',
  'title': 'Sample Title 3'}]


In [8]:
def insert_document(insert_document):
    return es.index(index=index_name, document=insert_document)

for document in dummy_data:
    document["document_id"] = uuid.uuid4()
    response = insert_document(document)
    pprint(response)



ObjectApiResponse({'_index': 'myindex', '_id': '7K8sAJkB-JJed6hyuSjG', '_version': 1, 'result': 'created', '_shards': {'total': 1, 'successful': 1, 'failed': 0}, '_seq_no': 0, '_primary_term': 1})
ObjectApiResponse({'_index': 'myindex', '_id': '7a8sAJkB-JJed6hyuyiD', '_version': 1, 'result': 'created', '_shards': {'total': 1, 'successful': 1, 'failed': 0}, '_seq_no': 1, '_primary_term': 1})
ObjectApiResponse({'_index': 'myindex', '_id': '7q8sAJkB-JJed6hyvCil', '_version': 1, 'result': 'created', '_shards': {'total': 1, 'successful': 1, 'failed': 0}, '_seq_no': 0, '_primary_term': 1})
